In [1]:
pip install torch

In [2]:
pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:00


In [3]:
from faker import Faker
import random
import torch
from transformers import DistilBertTokenizer, DistilBertConfig,DistilBertForSequenceClassification,AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # Import tqdm for the loading bar

In [4]:
# Check if a GPU is available
if torch.cuda.is_available():
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu_name}")
else:
    print("No GPU available.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU: Tesla T4


In [5]:
def load_data_from_txt(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()
        names = []
        labels = []
        for line in lines:
          try:
            name, label = line.strip().split(', ')
            names.append(name)
            labels.append(int(label))
          except:
            continue
    return names, labels

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### TRAINING I

In [ ]:
def generate_two_worded(x,y):
    # Generate two random words
    word1 = ''.join(random.choices('abcdefghijklmnñopqrstuvwxyz', k=random.randint(x,y)))
    word2 = ''.join(random.choices('abcdefghijklmnñopqrstuvwxyz', k=random.randint(3, 10)))

    # Concatenate the two words with a space in between
    two_worded_string = f"{word1} {word2}"

    return two_worded_string

In [ ]:
from faker import Factory
faker = Factory.create('es_ES')

# Generate fake names and random words
fake_nameandsurname = [faker.name() for _ in range(500000)]
fake_name = [faker.first_name() for _ in range(500000)]
two_worded_strings = [generate_two_worded(3,10) for _ in range(300000)]
two_worded_strings2 = [generate_two_worded(5,8) for _ in range(200000)]

single_random_words = [''.join(random.choices('abcdefghijklmnñopqrstuvwxyz', k=random.randint(3, 10))) for _ in range(300000)]
single_random_words2 = [''.join(random.choices('abcdefghijklmnñopqrstuvwxyz', k=random.randint(5, 8))) for _ in range(200000)]


random_words = two_worded_strings + two_worded_strings2 + single_random_words + single_random_words2
fake_names = fake_nameandsurname + fake_name

random_words = list(set(random_words))

# Combine fake names and random words
all_names = fake_names + random_words



# Label the data (1 for valid names, 0 for invalid names)
labels = [1] * len(fake_names) + [0] * len(random_words)

# Shuffle the data
data = list(zip(all_names, labels))
random.shuffle(data)
names, labels = zip(*data)

# Example: Display the first 10 names and labels
for name, label in zip(names[:10], labels[:10]):
    print(f"Name: {name}, Label: {label}")

Name: Gilberto, Label: 1
Name: mmwtredp ossfqr, Label: 0
Name: wlbycgjshd aipqwñ, Label: 0
Name: Anselmo Perera Díaz, Label: 1
Name: Magdalena, Label: 1
Name: Jose Angel Garay Bravo, Label: 1
Name: dntbt, Label: 0
Name: kyzwupiñzj, Label: 0
Name: rtlghf bmlih, Label: 0
Name: tjmcizii, Label: 0


In [ ]:
print(len(labels))

1978055


In [ ]:
print(len(all_names))

1978055


In [ ]:
# Step 2: Load Pre-trained DistilBERT Model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
inputs = tokenizer(names, return_tensors='pt', padding=True, truncation=True).to(device)

In [ ]:
# Step 5: Fine-tune the Model
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
model.to(device)

In [ ]:
# Split the data into training and validation sets
train_names, val_names, train_labels, val_labels = train_test_split(names, labels, test_size=0.2, random_state=42)

In [ ]:
# Tokenize validation set
val_inputs = tokenizer(val_names, return_tensors='pt', padding=True, truncation=True).to(device)
val_dataset = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], torch.tensor(val_labels))
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)  # No need to shuffle for validation

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # You can experiment with different learning rates

num_epochs = 4  # Adjust as needed

# Training Loop with Validation
for epoch in range(num_epochs):
    model.train()

    tqdm_dataloader = tqdm(dataloader, desc=f"Epoch {epoch + 1}", position=0, leave=True)

    for batch in tqdm_dataloader:
        input_ids, attention_mask, target_labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=target_labels.to(device))
        loss = outputs.loss
        logits = outputs.logits  # Ensure we are correctly accessing logits
        loss.backward()
        optimizer.step()

        tqdm_dataloader.set_postfix({"Loss": loss.item()}, refresh=True)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0

    with torch.no_grad():
        for val_batch in val_dataloader:
            val_input_ids, val_attention_mask, val_target_labels = val_batch
            val_outputs = model(val_input_ids.to(device), attention_mask=val_attention_mask.to(device), labels=val_target_labels.to(device))
            val_loss += val_outputs.loss.item()
            val_logits = val_outputs.logits  # Ensure we are correctly accessing logits

            # Calculate accuracy on validation set
            predictions = torch.argmax(val_logits, dim=1)
            val_correct += (predictions == val_target_labels.to(device)).sum().item()

    val_loss /= len(val_dataloader)
    val_accuracy = val_correct / len(val_dataset)

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


Epoch 1: 100%|██████████| 15452/15452 [53:38<00:00,  4.80it/s, Loss=2.32e-5]


Epoch 1, Loss: 2.318282531632576e-05, Validation Loss: 0.00017808764371495518, Validation Accuracy: 0.9999848314010223


Epoch 2: 100%|██████████| 15452/15452 [53:41<00:00,  4.80it/s, Loss=7.6e-6]


Epoch 2, Loss: 7.60480179451406e-06, Validation Loss: 9.689525395345451e-05, Validation Accuracy: 0.9999823033011928


Epoch 3: 100%|██████████| 15452/15452 [53:39<00:00,  4.80it/s, Loss=5.2e-5]


Epoch 3, Loss: 5.197570862947032e-05, Validation Loss: 0.00015918423355875373, Validation Accuracy: 0.9999848314010223


Epoch 4: 100%|██████████| 15452/15452 [53:39<00:00,  4.80it/s, Loss=2.27e-5]


Epoch 4, Loss: 2.267295894853305e-05, Validation Loss: 0.00015567554060129594, Validation Accuracy: 0.9999823033011928


In [ ]:
# Step 6: Inference
new_names = ["Isabella","Maya","Ivy William","Elena ","asdas asdas","Muerta","Jade Benson"]
new_inputs = tokenizer(new_names, return_tensors='pt', padding=True, truncation=True)

with torch.no_grad():
    model.eval()
    outputs = model(**new_inputs.to(device))

predictions = torch.argmax(outputs.logits, dim=1).tolist()
print("Predictions for new names:", predictions)

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Assuming model is your DistilBERT model
model.save_pretrained("/content/drive/My Drive/distilbert_model_ES")
tokenizer.save_pretrained("/content/drive/My Drive/distilbert_model_ES")

('/content/drive/My Drive/distilbert_model_ES/tokenizer_config.json',
 '/content/drive/My Drive/distilbert_model_ES/special_tokens_map.json',
 '/content/drive/My Drive/distilbert_model_ES/vocab.txt',
 '/content/drive/My Drive/distilbert_model_ES/added_tokens.json')

### TRAINING II

In [ ]:
# Load the model and tokenizer
model_path = '/content/drive/My Drive/distilbert_model_ES'
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

In [ ]:
# Now you can use the model and tokenizer for inference or further training
text = "55996930481"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

# Convert probabilities to binary predictions with a 0.5 threshold
predictions = torch.argmax(logits, dim=1).tolist()

print(predictions)  # This will print a tensor of 0s and 1s

In [ ]:
model.eval()

In [ ]:
model.to(device)

In [ ]:
# Load your data
file_path1 = '/content/drive/My Drive/spanish_results3.txt'  # Replace with your file path
names1, labels1 = load_data_from_txt(file_path1)

In [ ]:
# Load your data
file_path2 = '/content/drive/My Drive/spanish_results2.txt'  # Replace with your file path
names2, labels2 = load_data_from_txt(file_path1)

In [ ]:
# Load your data
file_path3 = '/content/drive/My Drive/spanish_results.txt'  # Replace with your file path
names3, labels3 = load_data_from_txt(file_path1)

In [ ]:
names = names1 + names2 + names3
labels = labels1 + labels2 + labels3

In [ ]:
# Tokenize the data
encodings = tokenizer(names, truncation=True, padding=True, return_tensors="pt")

In [ ]:
# Step 5: Fine-tune the Model
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # You can experiment with different learning rates

num_epochs = 4  # Adjust as needed

# Training Loop with Validation
for epoch in range(num_epochs):
    model.train()

    tqdm_dataloader = tqdm(dataloader, desc=f"Epoch {epoch + 1}", position=0, leave=True)

    for batch in tqdm_dataloader:
        input_ids, attention_mask, target_labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=target_labels.to(device))
        loss = outputs.loss
        logits = outputs.logits  # Ensure we are correctly accessing logits
        loss.backward()
        optimizer.step()

        tqdm_dataloader.set_postfix({"Loss": loss.item()}, refresh=True)


    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Assuming model is your DistilBERT model
model.save_pretrained("/content/drive/My Drive/distilbert_model_ES_2")
tokenizer.save_pretrained("/content/drive/My Drive/distilbert_model_ES_2")

### TRAINING III

In [ ]:
# Load the model and tokenizer
model_path = '/content/drive/My Drive/distilbert_model_ES_2'
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

In [ ]:
model.eval()

In [ ]:
model.to(device)

In [ ]:
# Load your data
file_path1 = '/content/drive/My Drive/classified_names_es.txt'  # Replace with your file path
names, labels = load_data_from_txt(file_path1)

In [ ]:
# Tokenize the data
encodings = tokenizer(names, truncation=True, padding=True, return_tensors="pt")

In [ ]:
# Step 5: Fine-tune the Model
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # You can experiment with different learning rates

num_epochs = 4  # Adjust as needed

# Training Loop with Validation
for epoch in range(num_epochs):
    model.train()

    tqdm_dataloader = tqdm(dataloader, desc=f"Epoch {epoch + 1}", position=0, leave=True)

    for batch in tqdm_dataloader:
        input_ids, attention_mask, target_labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=target_labels.to(device))
        loss = outputs.loss
        logits = outputs.logits  # Ensure we are correctly accessing logits
        loss.backward()
        optimizer.step()

        tqdm_dataloader.set_postfix({"Loss": loss.item()}, refresh=True)


    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1: 100%|██████████| 3/3 [02:29<00:00, 49.99s/it, Loss=0.562]


Epoch 1, Loss: 0.5621964335441589


Epoch 2: 100%|██████████| 3/3 [02:21<00:00, 47.08s/it, Loss=0.269]


Epoch 2, Loss: 0.26890552043914795


Epoch 3: 100%|██████████| 3/3 [02:21<00:00, 47.26s/it, Loss=0.392]


Epoch 3, Loss: 0.3924938440322876


Epoch 4: 100%|██████████| 3/3 [02:20<00:00, 46.78s/it, Loss=0.248]

Epoch 4, Loss: 0.24815206229686737


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Assuming model is your DistilBERT model
model.save_pretrained("/content/drive/My Drive/distilbert_model_ES_3")
tokenizer.save_pretrained("/content/drive/My Drive/distilbert_model_ES_3")

('/content/drive/My Drive/distilbert_model_ES_3/tokenizer_config.json',
 '/content/drive/My Drive/distilbert_model_ES_3/special_tokens_map.json',
 '/content/drive/My Drive/distilbert_model_ES_3/vocab.txt',
 '/content/drive/My Drive/distilbert_model_ES_3/added_tokens.json')

### TRAINING IV

In [8]:
# Load the model and tokenizer
model_path = '/content/drive/My Drive/distilbert_model_ES_3'
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

In [9]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [17]:
# Load your data
file_path1 = '/content/drive/My Drive/spanish_results_4.txt'  # Replace with your file path
names1, labels1 = load_data_from_txt(file_path1)
# Load your data
file_path1 = '/content/drive/My Drive/spanish_results4.2.txt'  # Replace with your file path
names2, labels2 = load_data_from_txt(file_path1)
# Load your data
file_path1 = '/content/drive/My Drive/spanish_results4.3.txt'  # Replace with your file path
names3, labels3 = load_data_from_txt(file_path1)
# Load your data
file_path1 = '/content/drive/My Drive/spanish_results4.4.txt'  # Replace with your file path
names4, labels4 = load_data_from_txt(file_path1)

In [18]:
names = names1 + names2 4 names3 + names4
labels = labels1 + labels2 + labels3 + labels4

SyntaxError: invalid syntax (<ipython-input-18-d7ae4bbc64fc>, line 1)

In [19]:
# Tokenize the data
encodings = tokenizer(names, truncation=True, padding=True, return_tensors="pt")

In [20]:
# Step 5: Fine-tune the Model
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [21]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [22]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # You can experiment with different learning rates

num_epochs = 4  # Adjust as needed

# Training Loop with Validation
for epoch in range(num_epochs):
    model.train()

    tqdm_dataloader = tqdm(dataloader, desc=f"Epoch {epoch + 1}", position=0, leave=True)

    for batch in tqdm_dataloader:
        input_ids, attention_mask, target_labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=target_labels.to(device))
        loss = outputs.loss
        logits = outputs.logits  # Ensure we are correctly accessing logits
        loss.backward()
        optimizer.step()

        tqdm_dataloader.set_postfix({"Loss": loss.item()}, refresh=True)


    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1: 100%|██████████| 165/165 [05:16<00:00,  1.92s/it, Loss=0.0158]


Epoch 1, Loss: 0.015755968168377876


Epoch 2: 100%|██████████| 165/165 [05:16<00:00,  1.92s/it, Loss=0.000419]


Epoch 2, Loss: 0.0004189266182947904


Epoch 3: 100%|██████████| 165/165 [05:15<00:00,  1.91s/it, Loss=1.54e-5]


Epoch 3, Loss: 1.535406590846833e-05


Epoch 4: 100%|██████████| 165/165 [05:16<00:00,  1.92s/it, Loss=0.000124]

Epoch 4, Loss: 0.00012360808614175767


In [23]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Assuming model is your DistilBERT model
model.save_pretrained("/content/drive/My Drive/distilbert_model_ES_4")
tokenizer.save_pretrained("/content/drive/My Drive/distilbert_model_ES_4")

('/content/drive/My Drive/distilbert_model_ES_4/tokenizer_config.json',
 '/content/drive/My Drive/distilbert_model_ES_4/special_tokens_map.json',
 '/content/drive/My Drive/distilbert_model_ES_4/vocab.txt',
 '/content/drive/My Drive/distilbert_model_ES_4/added_tokens.json')